<a href="https://colab.research.google.com/github/HarshESC/TalkData-Mobile-Demographics/blob/main/Supervised_Learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#Imports
import numpy as np
import pandas as pd
from sklearn.datasets import load_boston
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.feature_selection import SelectKBest, f_regression, chi2
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score, recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import classification_report
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn import neighbors
from sklearn import svm
from sklearn.svm import SVC

from sklearn import tree
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.decomposition import PCA
from sklearn import ensemble

import warnings
warnings.filterwarnings('ignore')
from google.colab import drive 
drive.mount('/content/gdrive')
import os
os.environ['KAGGLE_CONFIG_DIR'] = "/content/gdrive/"
%cd "/content/gdrive/My Drive/Final Capstone"


Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
/content/gdrive/My Drive/Final Capstone


# Small EDA

In [2]:
master_df = pd.read_csv("master_clean.csv")

master_df.head()

,event_id,app_id,is_active,device_id,label_id_x,download_id,251,252,253,254,256,262,263,302,303,306,326,405,406,407,548,549,551,558,562,564,565,566,english_phone_brand_asus,english_phone_brand_coolpad,english_phone_brand_hammer,english_phone_brand_huawei,english_phone_brand_lshi,english_phone_brand_meizu,english_phone_brand_oppo,english_phone_brand_qiku,english_phone_brand_samsung,english_phone_brand_vivo,english_phone_brand_xiaomi,device_model_Galaxy Note 3,device_model_Galaxy Note 4,device_model_Galaxy S5,device_model_Galaxy S6,device_model_MI 3,device_model_MI 4,device_model_MX4,device_model_MX4 Pro,device_model_MX5,device_model_Mate 7,device_model_P8 Lite,device_model_P8max,device_model_R7 Plus,device_model_U3,device_model_X5SL,device_model_ZenFone 2,device_model_note顶配版,device_model_坚果手机,device_model_大神Note3,device_model_小米4C,device_model_小米note,device_model_红米2,device_model_红米2A,device_model_红米Note2,device_model_红米Note3,device_model_红米note,device_model_荣耀3C,device_model_荣耀6,device_model_荣耀6 Plus,device_model_荣耀7,device_model_荣耀7i,device_model_荣耀X2,device_model_荣耀畅玩4C,device_model_荣耀畅玩5X,device_model_超级手机1,device_model_超级手机1 Pro,device_model_超级手机1s,device_model_青春版,device_model_魅蓝NOTE,device_model_魅蓝metal,num_group
0,198,-8807740666788515175,0,6984152891252102027,548,-1823587775536413148,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,4
1,198,-7415261454408512527,0,6984152891252102027,254,-431108563156410500,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,4
2,198,-6538188762242952077,1,6984152891252102027,251,445964129009149950,1,0,0,0,0,1,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,4
3,198,-6493194103110420302,0,6984152891252102027,253,490958788141681725,0,0,1,0,0,0,0,0,0,0,0,1,0,0,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,4
4,198,-6349602347932703147,0,6984152891252102027,405,634550543319398880,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,4


In [3]:
# Dropping AppID as I don't think it will be helpful for our question
master_df = master_df.drop(columns=["app_id"])


In [4]:

master_df["num_group"].value_counts()


5     2842
4     2690
3     1366
1     1250
2      885
11     857
10     683
0      637
9      403
8      282
6      194
7      142
Name: num_group, dtype: int64

In [5]:
numerical_columns = ["device_id", "event_id"]


# Select K Best

In [6]:
# splitting up the data and choosing num_group as target variable
X = master_df.drop(columns=["num_group"])
y = master_df["num_group"]

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.20, random_state=13, stratify=y
)


In [7]:
# set up preprocessing for pipeline (used to scale features)
preprocessing = ColumnTransformer(
    [("scale", StandardScaler(), numerical_columns),], remainder="passthrough",
)



## Random Forest Classifier

In [8]:
pipeline = Pipeline(
    [("preprocessing", preprocessing), ("rf", RandomForestClassifier())], verbose=True
)

pipeline.fit(X_train, y_train)


[Pipeline] ..... (step 1 of 2) Processing preprocessing, total=   0.0s
[Pipeline] ................ (step 2 of 2) Processing rf, total=   1.7s


Pipeline(memory=None,
         steps=[('preprocessing',
                 ColumnTransformer(n_jobs=None, remainder='passthrough',
                                   sparse_threshold=0.3,
                                   transformer_weights=None,
                                   transformers=[('scale',
                                                  StandardScaler(copy=True,
                                                                 with_mean=True,
                                                                 with_std=True),
                                                  ['device_id', 'event_id'])],
                                   verbose=False)),
                ('rf',
                 RandomForestClassifier(bootstrap=True, ccp_alpha=0.0,
                                        class_weight=None, criterion='gini',
                                        max_depth=None, max_features='auto',
                                        max_leaf_nodes=None, max_samples=None

Grid search

In [9]:
# Max depth of 50 provides a better score, this is much better than the previous score of around 0.2.
# To improve score suggest increasing max depth even more
grid = {
    "rf__max_depth": [50, 70, 90, 110],
    "rf__n_estimators": [1, 10, 100],
    "rf__min_samples_leaf": [1, 3, 5, 7],
    "rf__criterion": ["gini"],
}

model = GridSearchCV(pipeline, grid, n_jobs=-1, verbose=True)
model.fit(X_train, y_train)


Fitting 5 folds for each of 48 candidates, totalling 240 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:   16.5s
[Parallel(n_jobs=-1)]: Done 196 tasks      | elapsed:  1.1min
[Parallel(n_jobs=-1)]: Done 240 out of 240 | elapsed:  1.4min finished


[Pipeline] ..... (step 1 of 2) Processing preprocessing, total=   0.0s
[Pipeline] ................ (step 2 of 2) Processing rf, total=   1.6s


GridSearchCV(cv=None, error_score=nan,
             estimator=Pipeline(memory=None,
                                steps=[('preprocessing',
                                        ColumnTransformer(n_jobs=None,
                                                          remainder='passthrough',
                                                          sparse_threshold=0.3,
                                                          transformer_weights=None,
                                                          transformers=[('scale',
                                                                         StandardScaler(copy=True,
                                                                                        with_mean=True,
                                                                                        with_std=True),
                                                                         ['device_id',
                                                                 

In [10]:
model.best_params_


{'rf__criterion': 'gini',
 'rf__max_depth': 90,
 'rf__min_samples_leaf': 1,
 'rf__n_estimators': 100}

In [11]:
# Model still appears to be extremely overfitting
train_score = model.score(X_train, y_train)
test_score = model.score(X_test, y_test)

print(f"Train score: {train_score}")
print(f"Test score: {test_score}")


Train score: 1.0
Test score: 0.981610134859011


In [12]:
y_pred = model.predict(X_test)

con_mat = pd.DataFrame(
    confusion_matrix(y_test, y_pred),
    index=[
        "Actual M22-",
        "Actual M23-26",
        "Actual M27-28",
        "Actual M29-31",
        "Actual M32-38",
        "Actual M39+",
        "Actual F23-",
        "Actual F24-26",
        "Actual F27-28",
        "Actual F29-32",
        "Actual F33-42",
        "Actual F43+",
    ],
    columns=[
        "Predicted M22-",
        "Predicted M23-26",
        "Predicted M27-28",
        "Predicted M29-31",
        "Predicted M32-38",
        "Predicted M39+",
        "Predicted F23-",
        "Predicted F24-26",
        "Predicted F27-28",
        "Predicted F29-32",
        "Predicted F33-42",
        "Predicted F43+",
    ],
)
con_mat.style.background_gradient(axis=None)




,Predicted M22-,Predicted M23-26,Predicted M27-28,Predicted M29-31,Predicted M32-38,Predicted M39+,Predicted F23-,Predicted F24-26,Predicted F27-28,Predicted F29-32,Predicted F33-42,Predicted F43+
Actual M22-,127,0,0,0,0,0,0,0,0,0,0,0
Actual M23-26,1,246,0,1,1,0,0,0,0,1,0,0
Actual M27-28,0,0,175,0,2,0,0,0,0,0,0,0
Actual M29-31,0,2,0,259,4,3,0,0,0,0,2,3
Actual M32-38,0,0,0,0,533,4,0,1,0,0,0,0
Actual M39+,0,0,1,1,1,565,0,0,0,1,0,0
Actual F23-,0,0,0,2,0,0,37,0,0,0,0,0
Actual F24-26,0,0,0,0,1,0,0,27,0,0,0,0
Actual F27-28,0,1,0,2,0,2,0,0,51,0,0,0
Actual F29-32,0,0,0,0,0,0,0,0,0,81,0,0


In [13]:
print("Classification Report:")
print()
print(classification_report(y_test, y_pred))

Classification Report:

              precision    recall  f1-score   support

           0       0.99      1.00      1.00       127
           1       0.99      0.98      0.99       250
           2       0.99      0.99      0.99       177
           3       0.97      0.95      0.96       273
           4       0.98      0.99      0.98       538
           5       0.98      0.99      0.99       569
           6       1.00      0.95      0.97        39
           7       0.93      0.96      0.95        28
           8       1.00      0.91      0.95        56
           9       0.98      1.00      0.99        81
          10       0.99      0.99      0.99       137
          11       0.98      0.97      0.97       172

    accuracy                           0.98      2447
   macro avg       0.98      0.97      0.98      2447
weighted avg       0.98      0.98      0.98      2447



#K-Nearest Neighbor Classification

In [14]:
pipeline = Pipeline(
    [("preprocessing", preprocessing), ("knn", KNeighborsClassifier())], verbose=True
)

pipeline.fit(X_train, y_train)


[Pipeline] ..... (step 1 of 2) Processing preprocessing, total=   0.0s
[Pipeline] ............... (step 2 of 2) Processing knn, total=   0.1s


Pipeline(memory=None,
         steps=[('preprocessing',
                 ColumnTransformer(n_jobs=None, remainder='passthrough',
                                   sparse_threshold=0.3,
                                   transformer_weights=None,
                                   transformers=[('scale',
                                                  StandardScaler(copy=True,
                                                                 with_mean=True,
                                                                 with_std=True),
                                                  ['device_id', 'event_id'])],
                                   verbose=False)),
                ('knn',
                 KNeighborsClassifier(algorithm='auto', leaf_size=30,
                                      metric='minkowski', metric_params=None,
                                      n_jobs=None, n_neighbors=5, p=2,
                                      weights='uniform'))],
         verbose=True)

In [15]:
grid = {
    "knn__n_neighbors": [50, 100, 150],
    "knn__weights": ["distance"],
    "knn__leaf_size": [1, 10, 20],
}

model = GridSearchCV(pipeline, grid, n_jobs=-1, verbose=2, cv=2)
model.fit(X_train, y_train)


Fitting 2 folds for each of 9 candidates, totalling 18 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  18 out of  18 | elapsed:   12.6s finished


[Pipeline] ..... (step 1 of 2) Processing preprocessing, total=   0.0s
[Pipeline] ............... (step 2 of 2) Processing knn, total=   0.2s


GridSearchCV(cv=2, error_score=nan,
             estimator=Pipeline(memory=None,
                                steps=[('preprocessing',
                                        ColumnTransformer(n_jobs=None,
                                                          remainder='passthrough',
                                                          sparse_threshold=0.3,
                                                          transformer_weights=None,
                                                          transformers=[('scale',
                                                                         StandardScaler(copy=True,
                                                                                        with_mean=True,
                                                                                        with_std=True),
                                                                         ['device_id',
                                                                    

In [16]:
# It seems like the higher the amount of n_neighbors the better the score, for next test, will increase n_neighbors
model.best_params_


{'knn__leaf_size': 1, 'knn__n_neighbors': 150, 'knn__weights': 'distance'}

In [17]:
# Model still appears to be extremely overfitting
train_score = model.score(X_train, y_train)
test_score = model.score(X_test, y_test)

print(f"Train score: {train_score}")
print(f"Test score: {test_score}")


Train score: 1.0
Test score: 0.22353902738046588


In [18]:
y_pred = model.predict(X_test)

con_mat = pd.DataFrame(
    confusion_matrix(y_test, y_pred),
    index=[
        "Actual M22-",
        "Actual M23-26",
        "Actual M27-28",
        "Actual M29-31",
        "Actual M32-38",
        "Actual M39+",
        "Actual F23-",
        "Actual F24-26",
        "Actual F27-28",
        "Actual F29-32",
        "Actual F33-42",
        "Actual F43+",
    ],
    columns=[
        "Predicted M22-",
        "Predicted M23-26",
        "Predicted M27-28",
        "Predicted M29-31",
        "Predicted M32-38",
        "Predicted M39+",
        "Predicted F23-",
        "Predicted F24-26",
        "Predicted F27-28",
        "Predicted F29-32",
        "Predicted F33-42",
        "Predicted F43+",
    ],
)
con_mat.style.background_gradient(axis=None)


,Predicted M22-,Predicted M23-26,Predicted M27-28,Predicted M29-31,Predicted M32-38,Predicted M39+,Predicted F23-,Predicted F24-26,Predicted F27-28,Predicted F29-32,Predicted F33-42,Predicted F43+
Actual M22-,18,8,2,6,34,46,3,0,0,1,4,5
Actual M23-26,8,38,11,21,68,78,1,1,5,4,8,7
Actual M27-28,2,11,25,8,54,56,2,0,2,1,10,6
Actual M29-31,10,23,18,33,83,76,2,0,4,5,14,5
Actual M32-38,15,34,26,45,180,175,4,2,6,6,18,27
Actual M39+,24,31,22,40,167,221,4,4,3,12,16,25
Actual F23-,0,4,2,3,12,11,5,1,0,1,0,0
Actual F24-26,1,2,0,4,9,5,0,3,0,0,0,4
Actual F27-28,3,5,1,5,12,19,0,1,3,1,4,2
Actual F29-32,4,6,4,3,20,34,1,0,2,1,4,2


In [19]:
print("Classification Report:")
print()
print(classification_report(y_test, y_pred))


Classification Report:

              precision    recall  f1-score   support

           0       0.19      0.14      0.16       127
           1       0.21      0.15      0.18       250
           2       0.20      0.14      0.17       177
           3       0.17      0.12      0.14       273
           4       0.25      0.33      0.29       538
           5       0.26      0.39      0.31       569
           6       0.22      0.13      0.16        39
           7       0.23      0.11      0.15        28
           8       0.11      0.05      0.07        56
           9       0.03      0.01      0.02        81
          10       0.10      0.07      0.08       137
          11       0.11      0.06      0.08       172

    accuracy                           0.22      2447
   macro avg       0.17      0.14      0.15      2447
weighted avg       0.20      0.22      0.21      2447



# Decision Tree Classifier

In [20]:
pipeline = Pipeline(
    [("preprocessing", preprocessing), ("dtc", DecisionTreeClassifier())], verbose=True
)

pipeline.fit(X_train, y_train)


[Pipeline] ..... (step 1 of 2) Processing preprocessing, total=   0.0s
[Pipeline] ............... (step 2 of 2) Processing dtc, total=   0.1s


Pipeline(memory=None,
         steps=[('preprocessing',
                 ColumnTransformer(n_jobs=None, remainder='passthrough',
                                   sparse_threshold=0.3,
                                   transformer_weights=None,
                                   transformers=[('scale',
                                                  StandardScaler(copy=True,
                                                                 with_mean=True,
                                                                 with_std=True),
                                                  ['device_id', 'event_id'])],
                                   verbose=False)),
                ('dtc',
                 DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None,
                                        criterion='gini', max_depth=None,
                                        max_features=None, max_leaf_nodes=None,
                                        min_impurity_decrease=0.0,
      

In [21]:
# Max depth of 50 provides a better score, this is much better than the previous score of around 0.2.
# To improve score suggest increasing max depth even more
grid = {'max_leaf_nodes': list(range(2, 100)), 'min_samples_split': [2, 3, 4]}


model = GridSearchCV(DecisionTreeClassifier(random_state=42), grid, verbose=1, cv=3)


In [22]:
model.fit(X_train, y_train)


Fitting 3 folds for each of 294 candidates, totalling 882 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 882 out of 882 | elapsed:   45.7s finished


GridSearchCV(cv=3, error_score=nan,
             estimator=DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None,
                                              criterion='gini', max_depth=None,
                                              max_features=None,
                                              max_leaf_nodes=None,
                                              min_impurity_decrease=0.0,
                                              min_impurity_split=None,
                                              min_samples_leaf=1,
                                              min_samples_split=2,
                                              min_weight_fraction_leaf=0.0,
                                              presort='deprecated',
                                              random_state=42,
                                              splitter='best'),
             iid='deprecated', n_jobs=None,
             param_grid={'max_leaf_nodes': [2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12,


In [23]:
model.best_params_


{'max_leaf_nodes': 99, 'min_samples_split': 2}

In [24]:
# Model still appears to be extremely overfitting
train_score = model.score(X_train, y_train)
test_score = model.score(X_test, y_test)

print(f"Train score: {train_score}")
print(f"Test score: {test_score}")


Train score: 0.8423957481602616
Test score: 0.8406211687780957


In [25]:
y_pred = model.predict(X_test)

con_mat = pd.DataFrame(
    confusion_matrix(y_test, y_pred),
    index=[
        "Actual M22-",
        "Actual M23-26",
        "Actual M27-28",
        "Actual M29-31",
        "Actual M32-38",
        "Actual M39+",
        "Actual F23-",
        "Actual F24-26",
        "Actual F27-28",
        "Actual F29-32",
        "Actual F33-42",
        "Actual F43+",
    ],
    columns=[
        "Predicted M22-",
        "Predicted M23-26",
        "Predicted M27-28",
        "Predicted M29-31",
        "Predicted M32-38",
        "Predicted M39+",
        "Predicted F23-",
        "Predicted F24-26",
        "Predicted F27-28",
        "Predicted F29-32",
        "Predicted F33-42",
        "Predicted F43+",
    ],
)
con_mat.style.background_gradient(axis=None)


,Predicted M22-,Predicted M23-26,Predicted M27-28,Predicted M29-31,Predicted M32-38,Predicted M39+,Predicted F23-,Predicted F24-26,Predicted F27-28,Predicted F29-32,Predicted F33-42,Predicted F43+
Actual M22-,93,0,5,0,18,0,0,0,0,0,7,4
Actual M23-26,0,219,0,0,16,11,0,0,0,0,0,4
Actual M27-28,0,5,139,6,21,6,0,0,0,0,0,0
Actual M29-31,0,0,8,184,18,63,0,0,0,0,0,0
Actual M32-38,0,0,0,17,502,12,0,0,7,0,0,0
Actual M39+,9,0,2,0,3,540,0,0,0,0,0,15
Actual F23-,0,0,0,0,10,0,29,0,0,0,0,0
Actual F24-26,0,0,0,0,8,0,0,20,0,0,0,0
Actual F27-28,0,11,0,8,14,0,0,0,23,0,0,0
Actual F29-32,0,0,0,8,0,0,0,0,0,73,0,0


In [26]:
print("Classification Report:")
print()
print(classification_report(y_test, y_pred))


Classification Report:

              precision    recall  f1-score   support

           0       0.91      0.73      0.81       127
           1       0.92      0.88      0.90       250
           2       0.86      0.79      0.82       177
           3       0.81      0.67      0.74       273
           4       0.81      0.93      0.87       538
           5       0.79      0.95      0.86       569
           6       1.00      0.74      0.85        39
           7       1.00      0.71      0.83        28
           8       0.77      0.41      0.53        56
           9       1.00      0.90      0.95        81
          10       0.94      0.78      0.85       137
          11       0.85      0.74      0.79       172

    accuracy                           0.84      2447
   macro avg       0.89      0.77      0.82      2447
weighted avg       0.85      0.84      0.84      2447



# Support Vector Classifier

In [27]:
pipeline = Pipeline(
    [("preprocessing", preprocessing), ("svcc", SVC())], verbose=True
)

pipeline.fit(X_train, y_train)


[Pipeline] ..... (step 1 of 2) Processing preprocessing, total=   0.0s
[Pipeline] .............. (step 2 of 2) Processing svcc, total=  21.5s


Pipeline(memory=None,
         steps=[('preprocessing',
                 ColumnTransformer(n_jobs=None, remainder='passthrough',
                                   sparse_threshold=0.3,
                                   transformer_weights=None,
                                   transformers=[('scale',
                                                  StandardScaler(copy=True,
                                                                 with_mean=True,
                                                                 with_std=True),
                                                  ['device_id', 'event_id'])],
                                   verbose=False)),
                ('svcc',
                 SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None,
                     coef0=0.0, decision_function_shape='ovr', degree=3,
                     gamma='scale', kernel='rbf', max_iter=-1,
                     probability=False, random_state=None, shrinking=True,
        

In [28]:
grid = { "svcc__gamma": [1.0, 5.0, 10.0], 'svcc__kernel': ['rbf']}


model = GridSearchCV(pipeline, grid, verbose=1, cv=3)

In [29]:
model.fit(X_train, y_train)


Fitting 3 folds for each of 3 candidates, totalling 9 fits
[Pipeline] ..... (step 1 of 2) Processing preprocessing, total=   0.0s


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[Pipeline] .............. (step 2 of 2) Processing svcc, total=  15.8s
[Pipeline] ..... (step 1 of 2) Processing preprocessing, total=   0.0s
[Pipeline] .............. (step 2 of 2) Processing svcc, total=  15.6s
[Pipeline] ..... (step 1 of 2) Processing preprocessing, total=   0.0s
[Pipeline] .............. (step 2 of 2) Processing svcc, total=  15.7s
[Pipeline] ..... (step 1 of 2) Processing preprocessing, total=   0.0s
[Pipeline] .............. (step 2 of 2) Processing svcc, total=  15.6s
[Pipeline] ..... (step 1 of 2) Processing preprocessing, total=   0.0s
[Pipeline] .............. (step 2 of 2) Processing svcc, total=  15.7s
[Pipeline] ..... (step 1 of 2) Processing preprocessing, total=   0.0s
[Pipeline] .............. (step 2 of 2) Processing svcc, total=  15.7s
[Pipeline] ..... (step 1 of 2) Processing preprocessing, total=   0.0s
[Pipeline] .............. (step 2 of 2) Processing svcc, total=  15.5s
[Pipeline] ..... (step 1 of 2) Processing preprocessing, total=   0.0s
[Pipel

[Parallel(n_jobs=1)]: Done   9 out of   9 | elapsed:  2.8min finished


[Pipeline] ..... (step 1 of 2) Processing preprocessing, total=   0.0s
[Pipeline] .............. (step 2 of 2) Processing svcc, total=  37.2s


GridSearchCV(cv=3, error_score=nan,
             estimator=Pipeline(memory=None,
                                steps=[('preprocessing',
                                        ColumnTransformer(n_jobs=None,
                                                          remainder='passthrough',
                                                          sparse_threshold=0.3,
                                                          transformer_weights=None,
                                                          transformers=[('scale',
                                                                         StandardScaler(copy=True,
                                                                                        with_mean=True,
                                                                                        with_std=True),
                                                                         ['device_id',
                                                                    

In [30]:
model.best_params_


{'svcc__gamma': 1.0, 'svcc__kernel': 'rbf'}

In [31]:
y_pred = model.predict(X_test)

con_mat = pd.DataFrame(
    confusion_matrix(y_test, y_pred),
    index=[
        "Actual M22-",
        "Actual M23-26",
        "Actual M27-28",
        "Actual M29-31",
        "Actual M32-38",
        "Actual M39+",
        "Actual F23-",
        "Actual F24-26",
        "Actual F27-28",
        "Actual F29-32",
        "Actual F33-42",
        "Actual F43+",
    ],
    columns=[
        "Predicted M22-",
        "Predicted M23-26",
        "Predicted M27-28",
        "Predicted M29-31",
        "Predicted M32-38",
        "Predicted M39+",
        "Predicted F23-",
        "Predicted F24-26",
        "Predicted F27-28",
        "Predicted F29-32",
        "Predicted F33-42",
        "Predicted F43+",
    ],
)
con_mat.style.background_gradient(axis=None)


,Predicted M22-,Predicted M23-26,Predicted M27-28,Predicted M29-31,Predicted M32-38,Predicted M39+,Predicted F23-,Predicted F24-26,Predicted F27-28,Predicted F29-32,Predicted F33-42,Predicted F43+
Actual M22-,0,0,0,0,0,127,0,0,0,0,0,0
Actual M23-26,0,0,0,0,0,250,0,0,0,0,0,0
Actual M27-28,0,0,0,0,0,177,0,0,0,0,0,0
Actual M29-31,0,0,0,0,0,273,0,0,0,0,0,0
Actual M32-38,0,0,0,0,0,538,0,0,0,0,0,0
Actual M39+,0,0,0,0,0,569,0,0,0,0,0,0
Actual F23-,0,0,0,0,0,39,0,0,0,0,0,0
Actual F24-26,0,0,0,0,0,28,0,0,0,0,0,0
Actual F27-28,0,0,0,0,0,56,0,0,0,0,0,0
Actual F29-32,0,0,0,0,0,81,0,0,0,0,0,0


In [32]:
# Model still appears to be extremely overfitting
train_score = model.score(X_train, y_train)
test_score = model.score(X_test, y_test)

print(f"Train score: {train_score}")
print(f"Test score: {test_score}")


Train score: 0.990290269828291
Test score: 0.23252962811606048


In [33]:
print("Classification Report:")
print()
print(classification_report(y_test, y_pred))


Classification Report:

              precision    recall  f1-score   support

           0       0.00      0.00      0.00       127
           1       0.00      0.00      0.00       250
           2       0.00      0.00      0.00       177
           3       0.00      0.00      0.00       273
           4       0.00      0.00      0.00       538
           5       0.23      1.00      0.38       569
           6       0.00      0.00      0.00        39
           7       0.00      0.00      0.00        28
           8       0.00      0.00      0.00        56
           9       0.00      0.00      0.00        81
          10       0.00      0.00      0.00       137
          11       0.00      0.00      0.00       172

    accuracy                           0.23      2447
   macro avg       0.02      0.08      0.03      2447
weighted avg       0.05      0.23      0.09      2447



# Gradient Boosting Classifier

In [34]:
pipeline = Pipeline(
    [("preprocessing", preprocessing), ("gb", GradientBoostingClassifier())], verbose=True
)

pipeline.fit(X_train, y_train)


[Pipeline] ..... (step 1 of 2) Processing preprocessing, total=   0.0s
[Pipeline] ................ (step 2 of 2) Processing gb, total=  30.4s


Pipeline(memory=None,
         steps=[('preprocessing',
                 ColumnTransformer(n_jobs=None, remainder='passthrough',
                                   sparse_threshold=0.3,
                                   transformer_weights=None,
                                   transformers=[('scale',
                                                  StandardScaler(copy=True,
                                                                 with_mean=True,
                                                                 with_std=True),
                                                  ['device_id', 'event_id'])],
                                   verbose=False)),
                ('gb',
                 GradientBoostingClassifier(ccp_alpha=0.0,
                                            criterion='friedman_mse', init=None,
                                            le...='deviance',
                                            max_depth=3, max_features=None,
                        

In [35]:
grid = {'gb__n_estimators': [100], 
          'gb__max_depth': [5],
          'gb__loss': ['deviance']}

model = GridSearchCV(pipeline, grid, verbose=1, cv=3)
model.fit(X_train, y_train)


Fitting 3 folds for each of 1 candidates, totalling 3 fits
[Pipeline] ..... (step 1 of 2) Processing preprocessing, total=   0.0s


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[Pipeline] ................ (step 2 of 2) Processing gb, total=  27.2s
[Pipeline] ..... (step 1 of 2) Processing preprocessing, total=   0.0s
[Pipeline] ................ (step 2 of 2) Processing gb, total=  27.0s
[Pipeline] ..... (step 1 of 2) Processing preprocessing, total=   0.0s
[Pipeline] ................ (step 2 of 2) Processing gb, total=  26.9s
[Pipeline] ..... (step 1 of 2) Processing preprocessing, total=   0.0s


[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:  1.4min finished


[Pipeline] ................ (step 2 of 2) Processing gb, total=  40.6s


GridSearchCV(cv=3, error_score=nan,
             estimator=Pipeline(memory=None,
                                steps=[('preprocessing',
                                        ColumnTransformer(n_jobs=None,
                                                          remainder='passthrough',
                                                          sparse_threshold=0.3,
                                                          transformer_weights=None,
                                                          transformers=[('scale',
                                                                         StandardScaler(copy=True,
                                                                                        with_mean=True,
                                                                                        with_std=True),
                                                                         ['device_id',
                                                                    

In [36]:
model.best_params_

{'gb__loss': 'deviance', 'gb__max_depth': 5, 'gb__n_estimators': 100}

In [37]:
# Model still appears to be extremely overfitting
train_score = model.score(X_train, y_train)
test_score = model.score(X_test, y_test)

print(f"Train score: {train_score}")
print(f"Test score: {test_score}")


Train score: 1.0
Test score: 1.0


In [38]:
y_pred = model.predict(X_test)

con_mat = pd.DataFrame(
    confusion_matrix(y_test, y_pred),
    index=[
        "Actual M22-",
        "Actual M23-26",
        "Actual M27-28",
        "Actual M29-31",
        "Actual M32-38",
        "Actual M39+",
        "Actual F23-",
        "Actual F24-26",
        "Actual F27-28",
        "Actual F29-32",
        "Actual F33-42",
        "Actual F43+",
    ],
    columns=[
        "Predicted M22-",
        "Predicted M23-26",
        "Predicted M27-28",
        "Predicted M29-31",
        "Predicted M32-38",
        "Predicted M39+",
        "Predicted F23-",
        "Predicted F24-26",
        "Predicted F27-28",
        "Predicted F29-32",
        "Predicted F33-42",
        "Predicted F43+",
    ],
)
con_mat.style.background_gradient(axis=None)


,Predicted M22-,Predicted M23-26,Predicted M27-28,Predicted M29-31,Predicted M32-38,Predicted M39+,Predicted F23-,Predicted F24-26,Predicted F27-28,Predicted F29-32,Predicted F33-42,Predicted F43+
Actual M22-,127,0,0,0,0,0,0,0,0,0,0,0
Actual M23-26,0,250,0,0,0,0,0,0,0,0,0,0
Actual M27-28,0,0,177,0,0,0,0,0,0,0,0,0
Actual M29-31,0,0,0,273,0,0,0,0,0,0,0,0
Actual M32-38,0,0,0,0,538,0,0,0,0,0,0,0
Actual M39+,0,0,0,0,0,569,0,0,0,0,0,0
Actual F23-,0,0,0,0,0,0,39,0,0,0,0,0
Actual F24-26,0,0,0,0,0,0,0,28,0,0,0,0
Actual F27-28,0,0,0,0,0,0,0,0,56,0,0,0
Actual F29-32,0,0,0,0,0,0,0,0,0,81,0,0


In [39]:
print("Classification Report:")
print()
print(classification_report(y_test, y_pred))


Classification Report:

              precision    recall  f1-score   support

           0       1.00      1.00      1.00       127
           1       1.00      1.00      1.00       250
           2       1.00      1.00      1.00       177
           3       1.00      1.00      1.00       273
           4       1.00      1.00      1.00       538
           5       1.00      1.00      1.00       569
           6       1.00      1.00      1.00        39
           7       1.00      1.00      1.00        28
           8       1.00      1.00      1.00        56
           9       1.00      1.00      1.00        81
          10       1.00      1.00      1.00       137
          11       1.00      1.00      1.00       172

    accuracy                           1.00      2447
   macro avg       1.00      1.00      1.00      2447
weighted avg       1.00      1.00      1.00      2447



# PCA Models

In [40]:
# Since I am performing pca its necessary to standardize data and fit it to data frame.
scaler = StandardScaler()
X_train_std = scaler.fit_transform(X_train)

# Computing the principal components
pca = PCA()
X_pca = pca.fit_transform(X_train_std)


In [41]:
# I plan to use the amount of components that add up to 90% variance.
n_components = np.sum(np.cumsum(pca.explained_variance_ratio_) < 0.90)
X_pca = X_pca[:, :n_components]


In [42]:
# check to see the amount of components being used
n_components


47

# Random Forest Classifier

In [43]:
pipeline = Pipeline(
    [
        ("scaler", StandardScaler()),
        ("pca", PCA(n_components=n_components)),
        ("rf", RandomForestClassifier()),
    ]
)
pipeline = pipeline.fit(X_train, y_train)


GridSearch

In [44]:
grid = {
    "rf__max_depth": [90, 110, 130],
    "rf__n_estimators": [1, 10, 100],
    "rf__min_samples_leaf": [1, 3, 5, 7],
    "rf__criterion": ["gini"],
}

model = GridSearchCV(pipeline, grid, n_jobs=-1, cv=2, verbose=2)
model.fit(X_train, y_train)


Fitting 2 folds for each of 36 candidates, totalling 72 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  37 tasks      | elapsed:   39.9s
[Parallel(n_jobs=-1)]: Done  72 out of  72 | elapsed:  1.3min finished


GridSearchCV(cv=2, error_score=nan,
             estimator=Pipeline(memory=None,
                                steps=[('scaler',
                                        StandardScaler(copy=True,
                                                       with_mean=True,
                                                       with_std=True)),
                                       ('pca',
                                        PCA(copy=True, iterated_power='auto',
                                            n_components=47, random_state=None,
                                            svd_solver='auto', tol=0.0,
                                            whiten=False)),
                                       ('rf',
                                        RandomForestClassifier(bootstrap=True,
                                                               ccp_alpha=0.0,
                                                               class_weight=None,
                                      

In [45]:
model.best_params_


{'rf__criterion': 'gini',
 'rf__max_depth': 130,
 'rf__min_samples_leaf': 3,
 'rf__n_estimators': 100}

In [46]:
# Model still appears to be extremely overfitting
train_score = model.score(X_train, y_train)
test_score = model.score(X_test, y_test)

print(f"Train score: {train_score}")
print(f"Test score: {test_score}")


Train score: 0.9648405560098119
Test score: 0.723334695545566


In [47]:
y_pred = model.predict(X_test)

con_mat = pd.DataFrame(
    confusion_matrix(y_test, y_pred),
    index=[
        "Actual M22-",
        "Actual M23-26",
        "Actual M27-28",
        "Actual M29-31",
        "Actual M32-38",
        "Actual M39+",
        "Actual F23-",
        "Actual F24-26",
        "Actual F27-28",
        "Actual F29-32",
        "Actual F33-42",
        "Actual F43+",
    ],
    columns=[
        "Predicted M22-",
        "Predicted M23-26",
        "Predicted M27-28",
        "Predicted M29-31",
        "Predicted M32-38",
        "Predicted M39+",
        "Predicted F23-",
        "Predicted F24-26",
        "Predicted F27-28",
        "Predicted F29-32",
        "Predicted F33-42",
        "Predicted F43+",
    ],
)
con_mat.style.background_gradient(axis=None)


,Predicted M22-,Predicted M23-26,Predicted M27-28,Predicted M29-31,Predicted M32-38,Predicted M39+,Predicted F23-,Predicted F24-26,Predicted F27-28,Predicted F29-32,Predicted F33-42,Predicted F43+
Actual M22-,85,6,1,6,11,18,0,0,0,0,0,0
Actual M23-26,3,194,7,8,11,21,0,0,0,2,1,3
Actual M27-28,3,16,139,0,6,9,0,0,0,0,2,2
Actual M29-31,8,7,5,175,33,35,0,0,1,0,4,5
Actual M32-38,5,1,2,21,419,58,0,4,0,5,5,18
Actual M39+,10,14,3,17,44,454,1,3,1,7,14,1
Actual F23-,0,0,0,4,4,9,20,0,0,0,2,0
Actual F24-26,2,0,0,1,3,6,0,14,0,0,2,0
Actual F27-28,0,6,0,7,5,7,0,0,30,1,0,0
Actual F29-32,2,0,0,0,5,12,0,0,0,59,3,0


In [48]:
print("Classification Report:")
print()
print(classification_report(y_test, y_pred))

Classification Report:

              precision    recall  f1-score   support

           0       0.71      0.67      0.69       127
           1       0.77      0.78      0.77       250
           2       0.85      0.79      0.82       177
           3       0.68      0.64      0.66       273
           4       0.72      0.78      0.75       538
           5       0.67      0.80      0.73       569
           6       0.87      0.51      0.65        39
           7       0.67      0.50      0.57        28
           8       0.94      0.54      0.68        56
           9       0.79      0.73      0.76        81
          10       0.71      0.64      0.67       137
          11       0.74      0.54      0.62       172

    accuracy                           0.72      2447
   macro avg       0.76      0.66      0.70      2447
weighted avg       0.73      0.72      0.72      2447



#K Nearest Neighbor Classifier

In [49]:
pipeline = Pipeline(
    [
        ("scaler", StandardScaler()),
        ("pca", PCA(n_components=n_components)),
        ("knn", KNeighborsClassifier()),
    ]
)
pipeline = pipeline.fit(X_train, y_train)


Grid Search

In [50]:
grid = {
    "knn__n_neighbors": [50, 100, 150],
    "knn__weights": ["distance"],
    "knn__leaf_size": [1, 10, 20],
}

model = GridSearchCV(pipeline, grid, n_jobs=-1, cv=2, verbose=2)
model.fit(X_train, y_train)


Fitting 2 folds for each of 9 candidates, totalling 18 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  18 out of  18 | elapsed:  1.7min finished


GridSearchCV(cv=2, error_score=nan,
             estimator=Pipeline(memory=None,
                                steps=[('scaler',
                                        StandardScaler(copy=True,
                                                       with_mean=True,
                                                       with_std=True)),
                                       ('pca',
                                        PCA(copy=True, iterated_power='auto',
                                            n_components=47, random_state=None,
                                            svd_solver='auto', tol=0.0,
                                            whiten=False)),
                                       ('knn',
                                        KNeighborsClassifier(algorithm='auto',
                                                             leaf_size=30,
                                                             metric='minkowski',
                                         

In [51]:
model.best_params_


{'knn__leaf_size': 1, 'knn__n_neighbors': 50, 'knn__weights': 'distance'}

In [52]:
# Model still appears to be extremely overfitting
train_score = model.score(X_train, y_train)
test_score = model.score(X_test, y_test)

print(f"Train score: {train_score}")
print(f"Test score: {test_score}")


Train score: 0.9986713000817662
Test score: 0.5725378013894564


In [53]:
y_pred = model.predict(X_test)

con_mat = pd.DataFrame(
    confusion_matrix(y_test, y_pred),
    index=[
        "Actually M22-",
        "Actually M23-26",
        "Actually M27-28",
        "Actually M29-31",
        "Actually M32-38",
        "Actually M39+",
        "Actually F23-",
        "Actually F24-26",
        "Actually F27-28",
        "Actually F29-32",
        "Actually F33-42",
        "Actually F43+",
    ],
    columns=[
        "Predicted M22-",
        "Predicted M23-26",
        "Predicted M27-28",
        "Predicted M29-31",
        "Predicted M32-38",
        "Predicted M39+",
        "Predicted F23-",
        "Predicted F24-26",
        "Predicted F27-28",
        "Predicted F29-32",
        "Predicted F33-42",
        "Predicted F43+",
    ],
)
con_mat.style.background_gradient(axis=None)


,Predicted M22-,Predicted M23-26,Predicted M27-28,Predicted M29-31,Predicted M32-38,Predicted M39+,Predicted F23-,Predicted F24-26,Predicted F27-28,Predicted F29-32,Predicted F33-42,Predicted F43+
Actually M22-,62,7,1,6,21,25,0,3,0,2,0,0
Actually M23-26,7,157,4,2,31,38,0,0,0,4,7,0
Actually M27-28,4,26,91,7,16,19,0,0,0,0,6,8
Actually M29-31,15,18,4,109,53,60,1,0,3,0,4,6
Actually M32-38,4,4,3,28,377,79,0,4,1,9,5,24
Actually M39+,20,23,1,19,79,398,2,1,0,8,14,4
Actually F23-,0,1,0,2,6,9,20,0,0,0,1,0
Actually F24-26,2,0,0,3,2,10,0,10,0,0,1,0
Actually F27-28,0,10,0,8,8,7,0,0,19,0,1,3
Actually F29-32,2,1,5,4,9,23,0,0,0,35,2,0


In [54]:
print("Classification Report:")
print()
print(classification_report(y_test, y_pred))

Classification Report:

              precision    recall  f1-score   support

           0       0.52      0.49      0.50       127
           1       0.62      0.63      0.63       250
           2       0.78      0.51      0.62       177
           3       0.54      0.40      0.46       273
           4       0.57      0.70      0.63       538
           5       0.53      0.70      0.60       569
           6       0.69      0.51      0.59        39
           7       0.56      0.36      0.43        28
           8       0.79      0.34      0.47        56
           9       0.59      0.43      0.50        81
          10       0.57      0.42      0.48       137
          11       0.58      0.38      0.46       172

    accuracy                           0.57      2447
   macro avg       0.61      0.49      0.53      2447
weighted avg       0.58      0.57      0.57      2447



# Decision Tree Classifier 

In [55]:
pipeline = Pipeline(
    [("preprocessing", preprocessing), ("dtc", DecisionTreeClassifier())], verbose=True
)

pipeline.fit(X_train, y_train)


[Pipeline] ..... (step 1 of 2) Processing preprocessing, total=   0.0s
[Pipeline] ............... (step 2 of 2) Processing dtc, total=   0.1s


Pipeline(memory=None,
         steps=[('preprocessing',
                 ColumnTransformer(n_jobs=None, remainder='passthrough',
                                   sparse_threshold=0.3,
                                   transformer_weights=None,
                                   transformers=[('scale',
                                                  StandardScaler(copy=True,
                                                                 with_mean=True,
                                                                 with_std=True),
                                                  ['device_id', 'event_id'])],
                                   verbose=False)),
                ('dtc',
                 DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None,
                                        criterion='gini', max_depth=None,
                                        max_features=None, max_leaf_nodes=None,
                                        min_impurity_decrease=0.0,
      

In [56]:
# Max depth of 50 provides a better score, this is much better than the previous score of around 0.2.
# To improve score suggest increasing max depth even more
grid = {'max_leaf_nodes': list(range(2, 100)), 'min_samples_split': [2, 3, 4]}


model = GridSearchCV(DecisionTreeClassifier(random_state=42), grid, verbose=1, cv=3)


In [57]:
model.fit(X_train, y_train)


Fitting 3 folds for each of 294 candidates, totalling 882 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 882 out of 882 | elapsed:   45.8s finished


GridSearchCV(cv=3, error_score=nan,
             estimator=DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None,
                                              criterion='gini', max_depth=None,
                                              max_features=None,
                                              max_leaf_nodes=None,
                                              min_impurity_decrease=0.0,
                                              min_impurity_split=None,
                                              min_samples_leaf=1,
                                              min_samples_split=2,
                                              min_weight_fraction_leaf=0.0,
                                              presort='deprecated',
                                              random_state=42,
                                              splitter='best'),
             iid='deprecated', n_jobs=None,
             param_grid={'max_leaf_nodes': [2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12,


In [58]:
model.best_params_

{'max_leaf_nodes': 99, 'min_samples_split': 2}

In [59]:
# Model still appears to be extremely overfitting
train_score = model.score(X_train, y_train)
test_score = model.score(X_test, y_test)

print(f"Train score: {train_score}")
print(f"Test score: {test_score}")


Train score: 0.8423957481602616
Test score: 0.8406211687780957


In [60]:
y_pred = model.predict(X_test)

con_mat = pd.DataFrame(
    confusion_matrix(y_test, y_pred),
    index=[
        "Actually M22-",
        "Actually M23-26",
        "Actually M27-28",
        "Actually M29-31",
        "Actually M32-38",
        "Actually M39+",
        "Actually F23-",
        "Actually F24-26",
        "Actually F27-28",
        "Actually F29-32",
        "Actually F33-42",
        "Actually F43+",
    ],
    columns=[
        "Predicted M22-",
        "Predicted M23-26",
        "Predicted M27-28",
        "Predicted M29-31",
        "Predicted M32-38",
        "Predicted M39+",
        "Predicted F23-",
        "Predicted F24-26",
        "Predicted F27-28",
        "Predicted F29-32",
        "Predicted F33-42",
        "Predicted F43+",
    ],
)
con_mat.style.background_gradient(axis=None)


,Predicted M22-,Predicted M23-26,Predicted M27-28,Predicted M29-31,Predicted M32-38,Predicted M39+,Predicted F23-,Predicted F24-26,Predicted F27-28,Predicted F29-32,Predicted F33-42,Predicted F43+
Actually M22-,93,0,5,0,18,0,0,0,0,0,7,4
Actually M23-26,0,219,0,0,16,11,0,0,0,0,0,4
Actually M27-28,0,5,139,6,21,6,0,0,0,0,0,0
Actually M29-31,0,0,8,184,18,63,0,0,0,0,0,0
Actually M32-38,0,0,0,17,502,12,0,0,7,0,0,0
Actually M39+,9,0,2,0,3,540,0,0,0,0,0,15
Actually F23-,0,0,0,0,10,0,29,0,0,0,0,0
Actually F24-26,0,0,0,0,8,0,0,20,0,0,0,0
Actually F27-28,0,11,0,8,14,0,0,0,23,0,0,0
Actually F29-32,0,0,0,8,0,0,0,0,0,73,0,0


In [61]:
print("Classification Report:")
print()
print(classification_report(y_test, y_pred))

Classification Report:

              precision    recall  f1-score   support

           0       0.91      0.73      0.81       127
           1       0.92      0.88      0.90       250
           2       0.86      0.79      0.82       177
           3       0.81      0.67      0.74       273
           4       0.81      0.93      0.87       538
           5       0.79      0.95      0.86       569
           6       1.00      0.74      0.85        39
           7       1.00      0.71      0.83        28
           8       0.77      0.41      0.53        56
           9       1.00      0.90      0.95        81
          10       0.94      0.78      0.85       137
          11       0.85      0.74      0.79       172

    accuracy                           0.84      2447
   macro avg       0.89      0.77      0.82      2447
weighted avg       0.85      0.84      0.84      2447



# Support Vector Classifier

In [62]:
pipeline = Pipeline(
    [("preprocessing", preprocessing), ("svcc", SVC())], verbose=True
)

pipeline.fit(X_train, y_train)


[Pipeline] ..... (step 1 of 2) Processing preprocessing, total=   0.0s
[Pipeline] .............. (step 2 of 2) Processing svcc, total=  22.1s


Pipeline(memory=None,
         steps=[('preprocessing',
                 ColumnTransformer(n_jobs=None, remainder='passthrough',
                                   sparse_threshold=0.3,
                                   transformer_weights=None,
                                   transformers=[('scale',
                                                  StandardScaler(copy=True,
                                                                 with_mean=True,
                                                                 with_std=True),
                                                  ['device_id', 'event_id'])],
                                   verbose=False)),
                ('svcc',
                 SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None,
                     coef0=0.0, decision_function_shape='ovr', degree=3,
                     gamma='scale', kernel='rbf', max_iter=-1,
                     probability=False, random_state=None, shrinking=True,
        

In [63]:
grid = { "svcc__gamma": [1.0, 5.0, 10.0], 'svcc__kernel': ['rbf']}


model = GridSearchCV(pipeline, grid, verbose=1, cv=3)

In [64]:
model.fit(X_train, y_train)


Fitting 3 folds for each of 3 candidates, totalling 9 fits
[Pipeline] ..... (step 1 of 2) Processing preprocessing, total=   0.0s


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[Pipeline] .............. (step 2 of 2) Processing svcc, total=  15.7s
[Pipeline] ..... (step 1 of 2) Processing preprocessing, total=   0.0s
[Pipeline] .............. (step 2 of 2) Processing svcc, total=  15.7s
[Pipeline] ..... (step 1 of 2) Processing preprocessing, total=   0.0s
[Pipeline] .............. (step 2 of 2) Processing svcc, total=  15.6s
[Pipeline] ..... (step 1 of 2) Processing preprocessing, total=   0.0s
[Pipeline] .............. (step 2 of 2) Processing svcc, total=  15.5s
[Pipeline] ..... (step 1 of 2) Processing preprocessing, total=   0.0s
[Pipeline] .............. (step 2 of 2) Processing svcc, total=  15.6s
[Pipeline] ..... (step 1 of 2) Processing preprocessing, total=   0.0s
[Pipeline] .............. (step 2 of 2) Processing svcc, total=  15.7s
[Pipeline] ..... (step 1 of 2) Processing preprocessing, total=   0.0s
[Pipeline] .............. (step 2 of 2) Processing svcc, total=  15.5s
[Pipeline] ..... (step 1 of 2) Processing preprocessing, total=   0.0s
[Pipel

[Parallel(n_jobs=1)]: Done   9 out of   9 | elapsed:  2.8min finished


[Pipeline] ..... (step 1 of 2) Processing preprocessing, total=   0.0s
[Pipeline] .............. (step 2 of 2) Processing svcc, total=  37.4s


GridSearchCV(cv=3, error_score=nan,
             estimator=Pipeline(memory=None,
                                steps=[('preprocessing',
                                        ColumnTransformer(n_jobs=None,
                                                          remainder='passthrough',
                                                          sparse_threshold=0.3,
                                                          transformer_weights=None,
                                                          transformers=[('scale',
                                                                         StandardScaler(copy=True,
                                                                                        with_mean=True,
                                                                                        with_std=True),
                                                                         ['device_id',
                                                                    

In [65]:
model.best_params_

{'svcc__gamma': 1.0, 'svcc__kernel': 'rbf'}

In [66]:
# Model still appears to be extremely overfitting
train_score = model.score(X_train, y_train)
test_score = model.score(X_test, y_test)

print(f"Train score: {train_score}")
print(f"Test score: {test_score}")


Train score: 0.990290269828291
Test score: 0.23252962811606048


In [67]:
y_pred = model.predict(X_test)

con_mat = pd.DataFrame(
    confusion_matrix(y_test, y_pred),
    index=[
        "Actually M22-",
        "Actually M23-26",
        "Actually M27-28",
        "Actually M29-31",
        "Actually M32-38",
        "Actually M39+",
        "Actually F23-",
        "Actually F24-26",
        "Actually F27-28",
        "Actually F29-32",
        "Actually F33-42",
        "Actually F43+",
    ],
    columns=[
        "Predicted M22-",
        "Predicted M23-26",
        "Predicted M27-28",
        "Predicted M29-31",
        "Predicted M32-38",
        "Predicted M39+",
        "Predicted F23-",
        "Predicted F24-26",
        "Predicted F27-28",
        "Predicted F29-32",
        "Predicted F33-42",
        "Predicted F43+",
    ],
)
con_mat.style.background_gradient(axis=None)


,Predicted M22-,Predicted M23-26,Predicted M27-28,Predicted M29-31,Predicted M32-38,Predicted M39+,Predicted F23-,Predicted F24-26,Predicted F27-28,Predicted F29-32,Predicted F33-42,Predicted F43+
Actually M22-,0,0,0,0,0,127,0,0,0,0,0,0
Actually M23-26,0,0,0,0,0,250,0,0,0,0,0,0
Actually M27-28,0,0,0,0,0,177,0,0,0,0,0,0
Actually M29-31,0,0,0,0,0,273,0,0,0,0,0,0
Actually M32-38,0,0,0,0,0,538,0,0,0,0,0,0
Actually M39+,0,0,0,0,0,569,0,0,0,0,0,0
Actually F23-,0,0,0,0,0,39,0,0,0,0,0,0
Actually F24-26,0,0,0,0,0,28,0,0,0,0,0,0
Actually F27-28,0,0,0,0,0,56,0,0,0,0,0,0
Actually F29-32,0,0,0,0,0,81,0,0,0,0,0,0


In [68]:
print("Classification Report:")
print()
print(classification_report(y_test, y_pred))

Classification Report:

              precision    recall  f1-score   support

           0       0.00      0.00      0.00       127
           1       0.00      0.00      0.00       250
           2       0.00      0.00      0.00       177
           3       0.00      0.00      0.00       273
           4       0.00      0.00      0.00       538
           5       0.23      1.00      0.38       569
           6       0.00      0.00      0.00        39
           7       0.00      0.00      0.00        28
           8       0.00      0.00      0.00        56
           9       0.00      0.00      0.00        81
          10       0.00      0.00      0.00       137
          11       0.00      0.00      0.00       172

    accuracy                           0.23      2447
   macro avg       0.02      0.08      0.03      2447
weighted avg       0.05      0.23      0.09      2447



# Gradient Boosting Classifier

In [69]:
pipeline = Pipeline(
    [("preprocessing", preprocessing), ("gb", GradientBoostingClassifier())], verbose=True
)

pipeline.fit(X_train, y_train)


[Pipeline] ..... (step 1 of 2) Processing preprocessing, total=   0.0s
[Pipeline] ................ (step 2 of 2) Processing gb, total=  30.3s


Pipeline(memory=None,
         steps=[('preprocessing',
                 ColumnTransformer(n_jobs=None, remainder='passthrough',
                                   sparse_threshold=0.3,
                                   transformer_weights=None,
                                   transformers=[('scale',
                                                  StandardScaler(copy=True,
                                                                 with_mean=True,
                                                                 with_std=True),
                                                  ['device_id', 'event_id'])],
                                   verbose=False)),
                ('gb',
                 GradientBoostingClassifier(ccp_alpha=0.0,
                                            criterion='friedman_mse', init=None,
                                            le...='deviance',
                                            max_depth=3, max_features=None,
                        

In [70]:
grid = {'gb__n_estimators': [100], 
          'gb__max_depth': [5],
          'gb__loss': ['deviance']}

model = GridSearchCV(pipeline, grid, verbose=1, cv=3)
model.fit(X_train, y_train)


Fitting 3 folds for each of 1 candidates, totalling 3 fits
[Pipeline] ..... (step 1 of 2) Processing preprocessing, total=   0.0s


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[Pipeline] ................ (step 2 of 2) Processing gb, total=  27.3s
[Pipeline] ..... (step 1 of 2) Processing preprocessing, total=   0.0s
[Pipeline] ................ (step 2 of 2) Processing gb, total=  27.3s
[Pipeline] ..... (step 1 of 2) Processing preprocessing, total=   0.0s
[Pipeline] ................ (step 2 of 2) Processing gb, total=  27.1s
[Pipeline] ..... (step 1 of 2) Processing preprocessing, total=   0.0s


[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:  1.4min finished


[Pipeline] ................ (step 2 of 2) Processing gb, total=  40.9s


GridSearchCV(cv=3, error_score=nan,
             estimator=Pipeline(memory=None,
                                steps=[('preprocessing',
                                        ColumnTransformer(n_jobs=None,
                                                          remainder='passthrough',
                                                          sparse_threshold=0.3,
                                                          transformer_weights=None,
                                                          transformers=[('scale',
                                                                         StandardScaler(copy=True,
                                                                                        with_mean=True,
                                                                                        with_std=True),
                                                                         ['device_id',
                                                                    

In [71]:
model.best_params_  

{'gb__loss': 'deviance', 'gb__max_depth': 5, 'gb__n_estimators': 100}

In [72]:
# Model still appears to be extremely overfitting
train_score = model.score(X_train, y_train)
test_score = model.score(X_test, y_test)

print(f"Train score: {train_score}")
print(f"Test score: {test_score}")


Train score: 1.0
Test score: 1.0


In [73]:
y_pred = model.predict(X_test)

con_mat = pd.DataFrame(
    confusion_matrix(y_test, y_pred),
    index=[
        "Actual M22-",
        "Actual M23-26",
        "Actual M27-28",
        "Actual M29-31",
        "Actual M32-38",
        "Actual M39+",
        "Actual F23-",
        "Actual F24-26",
        "Actual F27-28",
        "Actual F29-32",
        "Actual F33-42",
        "Actual F43+",
    ],
    columns=[
        "Predicted M22-",
        "Predicted M23-26",
        "Predicted M27-28",
        "Predicted M29-31",
        "Predicted M32-38",
        "Predicted M39+",
        "Predicted F23-",
        "Predicted F24-26",
        "Predicted F27-28",
        "Predicted F29-32",
        "Predicted F33-42",
        "Predicted F43+",
    ],
)
con_mat.style.background_gradient(axis=None)


,Predicted M22-,Predicted M23-26,Predicted M27-28,Predicted M29-31,Predicted M32-38,Predicted M39+,Predicted F23-,Predicted F24-26,Predicted F27-28,Predicted F29-32,Predicted F33-42,Predicted F43+
Actual M22-,127,0,0,0,0,0,0,0,0,0,0,0
Actual M23-26,0,250,0,0,0,0,0,0,0,0,0,0
Actual M27-28,0,0,177,0,0,0,0,0,0,0,0,0
Actual M29-31,0,0,0,273,0,0,0,0,0,0,0,0
Actual M32-38,0,0,0,0,538,0,0,0,0,0,0,0
Actual M39+,0,0,0,0,0,569,0,0,0,0,0,0
Actual F23-,0,0,0,0,0,0,39,0,0,0,0,0
Actual F24-26,0,0,0,0,0,0,0,28,0,0,0,0
Actual F27-28,0,0,0,0,0,0,0,0,56,0,0,0
Actual F29-32,0,0,0,0,0,0,0,0,0,81,0,0


In [74]:
print("Classification Report:")
print()
print(classification_report(y_test, y_pred))

Classification Report:

              precision    recall  f1-score   support

           0       1.00      1.00      1.00       127
           1       1.00      1.00      1.00       250
           2       1.00      1.00      1.00       177
           3       1.00      1.00      1.00       273
           4       1.00      1.00      1.00       538
           5       1.00      1.00      1.00       569
           6       1.00      1.00      1.00        39
           7       1.00      1.00      1.00        28
           8       1.00      1.00      1.00        56
           9       1.00      1.00      1.00        81
          10       1.00      1.00      1.00       137
          11       1.00      1.00      1.00       172

    accuracy                           1.00      2447
   macro avg       1.00      1.00      1.00      2447
weighted avg       1.00      1.00      1.00      2447

